In [14]:
# main_folder = '/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_testing'
# main_folder = '/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_training'
# main_folder = '/home/pgajo/working/incels/ECIR/results/ECIR_union_fixed_training'
# main_folder = '/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_testing'
# main_folder = '/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_training'
# main_folder = '/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_union_fixed_training'
main_folder = '/home/pgajo/working/incels/ECIR/results_fixed'

padding_right_modifier = 10
padding_left_modifier = 8
legend_height = 1.4
import os
import glob
import pandas as pd

# Create an empty list to store DataFrames
dfs = []
# Iterate over all subfolders in the main folder
for root, dirs, files in os.walk(main_folder):
    # Iterate over all CSV files in the subfolder
    for file in glob.glob(os.path.join(root, '*.csv')):
        # Check if the filename starts with 'ECIR'
        if os.path.basename(file).startswith('ECIR'):
            # Read the CSV file into a DataFrame
            temp_df = pd.read_csv(file)
            temp_df['dir'] = os.path.dirname(file)
            # print('temp_df', temp_df)
            # Add the DataFrame to the list
            dfs.append(temp_df)

# drop model column
for df in dfs:
    df.drop(columns=['model', 'train_set(s)', 'dev_set(s)', 'test_set(s)'], inplace=True)

# within column 'train_source', sub IFD-EN-5203_truncated_0.tsv with 0
for df in dfs:
    df['train_source'] = df['train_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['train_source'] = df['train_source'].str.replace('.tsv', '')
    df['dev_source'] = df['dev_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['dev_source'] = df['dev_source'].str.replace('.tsv', '')
    df['test_source'] = df['test_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['test_source'] = df['test_source'].str.replace('.tsv', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_testing', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_union_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_testing', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_union_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_fixed', '')
    
print('len(dfs)', len(dfs))
import os
import pandas as pd

# Step 1: Concatenate all individual dataframes into one
concatenated_df = pd.concat(dfs, ignore_index=True)
# Step 2: Group by 'dir' and 'epoch', then compute the average 'test_f1' for each group
grouped_df = concatenated_df.groupby(['dir', 'epoch'])['test_f1'].mean().reset_index()
# get st.dev
grouped_df['test_f1_stdev'] = concatenated_df.groupby(['dir', 'epoch'])['test_f1'].std().reset_index()['test_f1']

# Step 3 & 4: Find the epoch with the highest average 'test_f1' for each directory
# and create a new DataFrame to store these results
final_results = pd.DataFrame()

for dir_name, group in grouped_df.groupby('dir'):
    max_row = pd.DataFrame(group.loc[group['test_f1'].idxmax()])
    final_results = pd.concat([final_results, max_row.T])

# Reset index for better readability
final_results.reset_index(drop=True, inplace=True)

# save to csv
display(final_results)
final_results.to_csv(os.path.join(main_folder, 'final_results.csv'), index=False)

len(dfs) 55


,dir,epoch,test_f1,test_f1_stdev
0,/combination_traindev=df_0_test=df_0/full/Ince...,4.0,0.872584,0.002968
1,/combination_traindev=df_0_test=df_1/full/Ince...,3.0,0.828992,0.007292
2,/combination_traindev=df_0_test=df_2/full/Ince...,1.0,0.818756,0.005391
3,/combination_traindev=df_1_test=df_0/full/Ince...,4.0,0.862309,0.006495
4,/combination_traindev=df_2_test=df_0/full/Ince...,3.0,0.864498,0.008112
5,/metrics_truncated_df0+df1+df2/full/incel-bert...,4.0,0.867995,0.005056
6,/metrics_truncated_df0+df1/full/incel-bert-bas...,1.0,0.869066,0.00464
7,/metrics_truncated_df0+df2/full/incel-bert-bas...,2.0,0.866189,0.007761


In [16]:
import os
import glob
import pandas as pd

# Initialize main_folder
main_folder = '/home/pgajo/working/incels/ECIR/results_fixed'

# Create an empty list to store DataFrames
dfs = []

# Iterate over all subfolders in the main folder
for root, dirs, files in os.walk(main_folder):
    for file in glob.glob(os.path.join(root, '*.csv')):
        if os.path.basename(file).startswith('ECIR'):
            temp_df = pd.read_csv(file)
            temp_df['dir'] = os.path.dirname(file)
            dfs.append(temp_df)

# Drop unnecessary columns
for df in dfs:
    df.drop(columns=['model', 'train_set(s)', 'dev_set(s)', 'test_set(s)'], inplace=True)
    # The existing code for column manipulation remains the same
# within column 'train_source', sub IFD-EN-5203_truncated_0.tsv with 0
for df in dfs:
    df['train_source'] = df['train_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['train_source'] = df['train_source'].str.replace('.tsv', '')
    df['dev_source'] = df['dev_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['dev_source'] = df['dev_source'].str.replace('.tsv', '')
    df['test_source'] = df['test_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['test_source'] = df['test_source'].str.replace('.tsv', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_testing', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_union_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_testing', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_union_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_fixed', '')
# Concatenate all dataframes
concatenated_df = pd.concat(dfs, ignore_index=True)

# Create a new DataFrame to store F1 scores for each directory and epoch
f1_scores_df = pd.DataFrame()

# Group by 'dir' and 'epoch', collect the test_f1 scores into a list
f1_scores_df = concatenated_df.groupby(['dir', 'epoch'])['test_f1'].apply(list).reset_index(name='f1_scores')

# Save the DataFrame containing all F1 scores to CSV
f1_scores_df.to_csv(os.path.join(main_folder, 'f1_scores.csv'), index=False)

# Display the DataFrame
display(f1_scores_df)


,dir,epoch,f1_scores
0,/combination_traindev=df_0_test=df_0/full/Ince...,1.0,"[0.8636363636363638, 0.8585526315789475, 0.857..."
1,/combination_traindev=df_0_test=df_0/full/Ince...,2.0,"[0.8298279158699808, 0.8710217755443885, 0.870..."
2,/combination_traindev=df_0_test=df_0/full/Ince...,3.0,"[0.8698517298187809, 0.8613376835236541, 0.871..."
3,/combination_traindev=df_0_test=df_0/full/Ince...,4.0,"[0.8714524207011686, 0.8667790893760541, 0.876..."
4,/combination_traindev=df_0_test=df_1/full/Ince...,1.0,"[0.8197879858657243, 0.813380281690141, 0.8142..."
5,/combination_traindev=df_0_test=df_1/full/Ince...,2.0,"[0.809437386569873, 0.8144144144144144, 0.8122..."
6,/combination_traindev=df_0_test=df_1/full/Ince...,3.0,"[0.8336134453781513, 0.8173913043478261, 0.826..."
7,/combination_traindev=df_0_test=df_1/full/Ince...,4.0,"[0.839041095890411, 0.8125, 0.82842287694974, ..."
8,/combination_traindev=df_0_test=df_2/full/Ince...,1.0,"[0.813380281690141, 0.8136200716845878, 0.8191..."
9,/combination_traindev=df_0_test=df_2/full/Ince...,2.0,"[0.8127208480565371, 0.7918968692449354, 0.812..."


In [17]:
import os
import glob
import pandas as pd
import numpy as np

# Initialize main_folder
main_folder = '/home/pgajo/working/incels/ECIR/results_fixed'

# Create an empty list to store DataFrames
dfs = []

# Iterate over all subfolders in the main folder
for root, dirs, files in os.walk(main_folder):
    for file in glob.glob(os.path.join(root, '*.csv')):
        if os.path.basename(file).startswith('ECIR'):
            temp_df = pd.read_csv(file)
            temp_df['dir'] = os.path.dirname(file)
            dfs.append(temp_df)

# Drop unnecessary columns
for df in dfs:
    df.drop(columns=['model', 'train_set(s)', 'dev_set(s)', 'test_set(s)'], inplace=True)
    # The existing code for column manipulation remains the same

# Concatenate all dataframes
concatenated_df = pd.concat(dfs, ignore_index=True)

# Create a new DataFrame to store F1 scores for each directory and epoch
f1_scores_df = pd.DataFrame()

# Group by 'dir' and 'epoch', collect the test_f1 scores into a list
f1_scores_df = concatenated_df.groupby(['dir', 'epoch'])['test_f1'].apply(list).reset_index(name='f1_scores')

# Calculate mean F1 scores for each epoch and directory
f1_scores_df['mean_f1'] = f1_scores_df['f1_scores'].apply(np.mean)

# Initialize an empty DataFrame to store the final results
final_f1_scores = pd.DataFrame()

# For each directory, find the epoch with the highest mean F1 score and store it
for dir_name, group in f1_scores_df.groupby('dir'):
    max_row = group.loc[group['mean_f1'].idxmax()]
    final_f1_scores = pd.concat([final_f1_scores, max_row.to_frame().T])

# Reset index for better readability
final_f1_scores.reset_index(drop=True, inplace=True)

# Display the DataFrame containing epochs with the highest mean F1 for each directory
display(final_f1_scores)

# Optionally save the DataFrame to CSV
final_f1_scores.to_csv(os.path.join(main_folder, 'final_f1_scores.csv'), index=False)
print('saved to:', os.path.join(main_folder, 'final_f1_scores.csv'))


,dir,epoch,f1_scores,mean_f1
0,/home/pgajo/working/incels/ECIR/results_fixed/...,4.0,"[0.8714524207011686, 0.8667790893760541, 0.876...",0.872584
1,/home/pgajo/working/incels/ECIR/results_fixed/...,3.0,"[0.8336134453781513, 0.8173913043478261, 0.826...",0.828992
2,/home/pgajo/working/incels/ECIR/results_fixed/...,1.0,"[0.813380281690141, 0.8136200716845878, 0.8191...",0.818756
3,/home/pgajo/working/incels/ECIR/results_fixed/...,4.0,"[0.8756218905472636, 0.8576214405360134, 0.856...",0.862309
4,/home/pgajo/working/incels/ECIR/results_fixed/...,3.0,"[0.85667215815486, 0.8585690515806988, 0.86666...",0.864498
5,/home/pgajo/working/incels/ECIR/results_fixed/...,4.0,"[0.8689883913764511, 0.8708609271523179, 0.874...",0.867995
6,/home/pgajo/working/incels/ECIR/results_fixed/...,1.0,"[0.875207986688852, 0.8671096345514949, 0.8633...",0.869066
7,/home/pgajo/working/incels/ECIR/results_fixed/...,2.0,"[0.8737201365187712, 0.8635578583765112, 0.854...",0.866189


saved to: /home/pgajo/working/incels/ECIR/results_fixed/final_f1_scores.csv


In [13]:
# Reset index for better readability
grouped_df.reset_index(drop=True, inplace=True)

# Display the DataFrame
display(grouped_df)

,dir,epoch,test_f1,test_f1_stdev
0,/home/pgajo/working/incels/ECIR/results_fixed/...,1.0,0.855883,0.005561
1,/home/pgajo/working/incels/ECIR/results_fixed/...,2.0,0.850857,0.021164
2,/home/pgajo/working/incels/ECIR/results_fixed/...,3.0,0.872282,0.005033
3,/home/pgajo/working/incels/ECIR/results_fixed/...,4.0,0.872584,0.002968
4,/home/pgajo/working/incels/ECIR/results_fixed/...,1.0,0.818279,0.009953
5,/home/pgajo/working/incels/ECIR/results_fixed/...,2.0,0.814707,0.004848
6,/home/pgajo/working/incels/ECIR/results_fixed/...,3.0,0.828992,0.007292
7,/home/pgajo/working/incels/ECIR/results_fixed/...,4.0,0.828056,0.010537
8,/home/pgajo/working/incels/ECIR/results_fixed/...,1.0,0.818756,0.005391
9,/home/pgajo/working/incels/ECIR/results_fixed/...,2.0,0.807867,0.008998
